# SCRAPING THE URLS

In [1]:
#importing the necessary packages
import os
import pandas as pd
from urllib.request import urlopen
import bs4
import requests, lxml
import re

In [2]:
#reading the excel file
df=pd.read_csv("D:\\python\\ass\\py_ass\\Amazon Scraping - Sheet1.csv")
df.head()

,Unnamed: 0,id,Asin,country
0,0,1,1015,de
1,1,2424796,1015,fr
2,2,2,000004458X,de
3,3,2424797,000004458X,fr
4,4,3,1002198,de


In [3]:
#dropping the unwanted column
df.drop(columns=["Unnamed: 0"],inplace=True)

In [4]:
#viewing the shape of the dataset
shape=df.shape
print("There are",shape[1],"columns and",shape[0],"records in the datset given")

There are 3 columns and 1000 records in the datset given


In [5]:
#cehcking for any missing values
df.isnull().sum()

id         0
Asin       0
country    0
dtype: int64

In [6]:
#checking if any duplicate value is present
df.duplicated().sum()

0

In [7]:
#checking for datatype
df.dtypes

id          int64
Asin       object
country    object
dtype: object

In [10]:
#creating the website link using countrycode and asin
headers = {"User-Agent":"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"}
search_link=[]
website_link=[]
u="https://www.amazon."

for i in range(len(df[:100])):
    #creating the product links
    url=u+str(df["country"][i])+"/dp/"+str(df["Asin"][i])
    search_link.append(url)
    
    #since the created product link is not valid, directly seraching it in google search 
    html = requests.get(f'https://www.google.com/search?q=' + url,headers=headers)
    #checking the page status
    if html.status_code !=404:
        soup = bs4.BeautifulSoup(html.text, 'html.parser')
        #fetching the 1st link after google search
        link_html=soup.select("a",class_="yuRUbf")
        if len(link_html)>4:
            link=link_html[4]
            reg=re.findall(r"https://www.amazon.+&amp;s",str(link))[0][:-6]
            website_link.append(reg)
        else:
            website_link.append("page not found")  

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [17]:
#storing the website link and searching link in a dataframe
frameout=pd.DataFrame()
out["Asin"]=df["Asin"][:100]
out["country"]=df["country"][:100]
out["search_link"]=search_link
out["website_link"]=website_link
out

,search_link,website_link,Asin,country
0,https://www.amazon.de/dp/1015,https://www.amazon.in/Comp-Cams-1015-1-Competi...,1015,de
1,https://www.amazon.fr/dp/1015,https://www.amazon.in/Comp-Cams-1015-1-Competi...,1015,fr
2,https://www.amazon.de/dp/000004458X,https://www.amazon.de/Old-Spice-Rasur-Creme-or...,000004458X,de
3,https://www.amazon.fr/dp/000004458X,https://www.amazon.in/Old-Spice-Shave-Cream-Or...,000004458X,fr
4,https://www.amazon.de/dp/1002198,https://www.amazon.de/-/en/Vacuum-Flask-Stainl...,1002198,de
...,...,...,...,...
95,https://www.amazon.fr/dp/1060619,https://www.amazon.in/OXO-Grips-Stainless-Stee...,1060619,fr
96,https://www.amazon.de/dp/1060694,https://www.amazon.de/-/en/LJ-Vickery-ebook/dp...,1060694,de
97,https://www.amazon.es/dp/1060694,https://www.amazon.es/1060694-DBS50E-S5EK01000...,1060694,es
98,https://www.amazon.fr/dp/1060694,https://www.amazon.fr/1060694-DBS50E-S5EK01000...,1060694,fr


In [27]:
#fiding product title
head=({"user-agent":"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7","Accept-Language":"en-US, en;q=0.5"})
product_title=[]
for i in range(len(out)):
    page=requests.get(out["website_link"][i],headers=head)
    soup=bs4.BeautifulSoup(page.content,"lxml")
    title=soup.find("span",attrs={"id":"productTitle"})
    if title!=None:
        title_value=title.string
        product_title.append(title_value.strip())
    else:
        title="No title found"
        product_title.append(title)

In [76]:
#storing the product title in a dataframe
out["product_title"]=product_title

In [29]:
#fiding the product price
head=({"user-agent":"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"})
product_price=[]
for i in range(len(out)):
    page=requests.get(out["website_link"][i],headers=head)
    soup=bs4.BeautifulSoup(page.content,"lxml")
    price=soup.find("span",class_="a-offscreen")
    if price!=None:
        product_price.append(price.text)
    else:
        product_price.append("currently unavailable")

In [31]:
#storing the product price in a dataframe
out["product_price"]=product_price

In [39]:
#scraping the product detail
head=({"user-agent":"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7","Accept-Language":"en-US, en;q=0.5"})
product_detail=[]
for i in range(len(out)):
    page=requests.get(out["website_link"][i],headers=head)
    soup=bs4.BeautifulSoup(page.content,"lxml")
    product=soup.find("ul",class_="a-unordered-list a-vertical a-spacing-mini")
    if product!=None:
        product_detail.append(product.get_text().replace("  ",","))
    else:
        product_detail.append("no details available")
                              

In [41]:
#storing the product detail in a dataframe
out["product_detail"]=product_detail

In [68]:
#scraping the product image
head=({"user-agent":"Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"})
product_image=[]
for i in range(len(out)):
    page=requests.get(out["website_link"][i],headers=head)
    soup=bs4.BeautifulSoup(page.content,"lxml")
    links=soup.find_all("img")
    if links!=None:
        product_image.append(links[0].get("src"))
    else:
        product_image.append("image found,image url not found")

In [81]:
#storing the product image in a dataframe
out["product_image"]=product_image

In [82]:
#viewing the final df 
out.head()

,search_link,website_link,Asin,country,product_title,product_price,product_detail,product_image
0,https://www.amazon.de/dp/1015,https://www.amazon.in/Comp-Cams-1015-1-Competi...,1015,de,Competition Cams 1015-1,currently unavailable,",Constructed from an aluminum alloy with tensi...",https://images-na.ssl-images-amazon.com/captch...
1,https://www.amazon.fr/dp/1015,https://www.amazon.in/Comp-Cams-1015-1-Competi...,1015,fr,Competition Cams 1015-1,currently unavailable,no details available,https://images-na.ssl-images-amazon.com/captch...
2,https://www.amazon.de/dp/000004458X,https://www.amazon.de/Old-Spice-Rasur-Creme-or...,000004458X,de,Old Spice Shaving Cream - 70 g (original) - Pa...,currently unavailable,no details available,https://images-na.ssl-images-amazon.com/captch...
3,https://www.amazon.fr/dp/000004458X,https://www.amazon.in/Old-Spice-Shave-Cream-Or...,000004458X,fr,Old Spice Shave Cream - 70 g (Original) - Pack...,₹145.00,no details available,//fls-eu.amazon.in/1/batch/1/OP/A21TJRUUN4KGV:...
4,https://www.amazon.de/dp/1002198,https://www.amazon.de/-/en/Vacuum-Flask-Stainl...,1002198,de,Vacuum Flask Stainless Steel Thermal Flask 100...,currently unavailable,no details available,https://images-na.ssl-images-amazon.com/captch...


In [75]:
#storing the scraped information in a json file
out.to_json("main_task.json")